### Train Models on the BikeShare Dataset

In [1]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import pandas as pd
import tensorflow as tf

from xgboost import XGBRegressor as xgb
from sklearn.model_selection import train_test_split

In [2]:
df = pd.read_csv("Data/hour.csv", header=0, sep=',')
df.head()

instant      dteday  season  yr  mnth  hr  holiday  weekday  workingday  \
0        1  2011-01-01       1   0     1   0        0        6           0   
1        2  2011-01-01       1   0     1   1        0        6           0   
2        3  2011-01-01       1   0     1   2        0        6           0   
3        4  2011-01-01       1   0     1   3        0        6           0   
4        5  2011-01-01       1   0     1   4        0        6           0   

   weathersit  temp   atemp   hum  windspeed  casual  registered  cnt  
0           1  0.24  0.2879  0.81        0.0       3          13   16  
1           1  0.22  0.2727  0.80        0.0       8          32   40  
2           1  0.22  0.2727  0.80        0.0       5          27   32  
3           1  0.24  0.2879  0.75        0.0       3          10   13  
4           1  0.24  0.2879  0.75        0.0       0           1    1

In [3]:
train_idx = np.where(df['dteday'].values < '2012-07-01')[0]
test_idx = np.where(df['dteday'].values >= '2012-07-01')[0]

In [4]:
df['dteday'] = [sorted(set(df['dteday'].values)).index(x) for x in df['dteday'].values]
df.head()

instant  dteday  season  yr  mnth  hr  holiday  weekday  workingday  \
0        1       0       1   0     1   0        0        6           0   
1        2       0       1   0     1   1        0        6           0   
2        3       0       1   0     1   2        0        6           0   
3        4       0       1   0     1   3        0        6           0   
4        5       0       1   0     1   4        0        6           0   

   weathersit  temp   atemp   hum  windspeed  casual  registered  cnt  
0           1  0.24  0.2879  0.81        0.0       3          13   16  
1           1  0.22  0.2727  0.80        0.0       8          32   40  
2           1  0.22  0.2727  0.80        0.0       5          27   32  
3           1  0.24  0.2879  0.75        0.0       3          10   13  
4           1  0.24  0.2879  0.75        0.0       0           1    1

In [5]:
Y = df['cnt'].values
X = df.drop(['cnt', 'instant', 'casual', 'registered'], axis=1)
print(X.head())
X = X.values

   dteday  season  yr  mnth  hr  holiday  weekday  workingday  weathersit  \
0       0       1   0     1   0        0        6           0           1   
1       0       1   0     1   1        0        6           0           1   
2       0       1   0     1   2        0        6           0           1   
3       0       1   0     1   3        0        6           0           1   
4       0       1   0     1   4        0        6           0           1   

   temp   atemp   hum  windspeed  
0  0.24  0.2879  0.81        0.0  
1  0.22  0.2727  0.80        0.0  
2  0.22  0.2727  0.80        0.0  
3  0.24  0.2879  0.75        0.0  
4  0.24  0.2879  0.75        0.0  


In [16]:
split_time = True
if split_time:
    X_train = X[train_idx]
    X_test  = X[test_idx]
    Y_train = Y[train_idx]
    Y_test  = Y[test_idx]
    X_query = X.copy()
else:
    X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.25)
    X_query = X.copy()

In [17]:
Y_mean = np.mean(Y_train)
Y_std  = np.std(Y_train)
Y_train = (Y_train - Y_mean) / Y_std
Y_test  = (Y_test  - Y_mean) / Y_std

X_means = np.mean(X_train, axis=0)
X_stds = np.std(X_train, axis=0)
X_train = (X_train - X_means) / X_stds
X_test = (X_test - X_means) / X_stds

In [56]:
# Confirm that there are interactions in this dataset by building XGBoost models of different depths.

xgb1 = xgb(max_depth=1, n_estimators=2000)
xgb1.fit(X_train, Y_train)
print(np.mean(np.square(Y_train - xgb1.predict(X_train))))
print(np.mean(np.square(Y_test - xgb1.predict(X_test))))

0.291825726095841
0.8398885664517867


In [57]:
xgb2 = xgb(max_depth=2, n_estimators=1000)
xgb2.fit(X_train, Y_train)
print(np.mean(np.square(Y_train - xgb2.predict(X_train))))
print(np.mean(np.square(Y_test - xgb2.predict(X_test))))

0.138394253908292
0.5167277562993031


In [58]:
xgb3 = xgb(max_depth=3, n_estimators=1000)
xgb3.fit(X_train, Y_train)
print(np.mean(np.square(Y_train - xgb3.predict(X_train))))
print(np.mean(np.square(Y_test - xgb3.predict(X_test))))

0.04979640057972824
0.29513703662409957


In [59]:
xgb4 = xgb(max_depth=4, n_estimators=1000)
xgb4.fit(X_train, Y_train)
print(np.mean(np.square(Y_train - xgb4.predict(X_train))))
print(np.mean(np.square(Y_test - xgb4.predict(X_test))))

0.02554988638705647
0.24579002239462971


In [30]:
print(np.var(Y_train))
print(np.var(Y_test))

1.0000000000000002
1.867103089861165


In [25]:
def build_model(activation, dropout_rate=0.0, n_hidden=128):
    print("Building model with {} activation and {:.3f} dropout".format(activation, dropout_rate))
    model = tf.keras.models.Sequential([
        tf.keras.layers.Flatten(input_shape=(13,)),
        tf.keras.layers.Dropout(dropout_rate),
        tf.keras.layers.Dense(n_hidden, activation=activation),
        tf.keras.layers.Dropout(dropout_rate),
        tf.keras.layers.Dense(n_hidden, activation=activation),
        tf.keras.layers.Dropout(dropout_rate),
        tf.keras.layers.Dense(n_hidden, activation=activation),
        tf.keras.layers.Dropout(dropout_rate),
        tf.keras.layers.Dense(n_hidden, activation=activation),
        tf.keras.layers.Dropout(dropout_rate),
        tf.keras.layers.Dense(1, activation=tf.identity)
    ])
    sgd = tf.keras.optimizers.Adam(lr=1e-3)
    if activation == 'sigmoid':
        model.compile(
                      loss='MSE',
                      metrics=['mse'],
                     optimizer=sgd)
    else:
        model.compile(
              loss='MSE',
              metrics=['mse'],
                     optimizer=sgd)
    return model

In [60]:
max_epoch = 500
with open("results/results_bikeshare_128.tsv", 'a') as results_file:
    for fit_iter in range(3):
        for dropout_rate in [0.0, 0.05, 0.125, 0.25, 0.375, 0.5]:
            model = build_model('sigmoid', dropout_rate, 128)
            epoch = 0
            while epoch < max_epoch:
                print(dropout_rate, fit_iter, epoch)
                pred_train = model(X_train, training=False).numpy()
                pred_test  = model(X_test, training=False).numpy()
                pred_query = model(X_query, training=False).numpy()
                xgb1 = xgb(max_depth=1, n_estimators=1000)
                xgb2 = xgb(max_depth=2, n_estimators=1000)
                xgb3 = xgb(max_depth=3, n_estimators=1000)

                xgb1.fit(X_test, pred_test)
                xgb1_preds = xgb1.predict(X_test)

                xgb2.fit(X_test, pred_test)
                xgb2_preds = xgb2.predict(X_test)

                xgb3.fit(X_test, pred_test)
                xgb3_preds = xgb3.predict(X_test)

                xgb1_var = np.var(xgb1_preds)
                xgb2_var = np.var(xgb1_preds - xgb2_preds)
                xgb3_var = np.var(xgb2_preds - xgb3_preds)

                train_mse = np.mean(np.square(Y_train - np.squeeze(model(X_train, training=False).numpy())))
                val_mse = np.mean(np.square(Y_test - np.squeeze(model(X_test, training=False).numpy())))

                print(xgb1_var, xgb2_var, xgb3_var, train_mse, val_mse)
                print('{:.3f}\t{:d}\t{:d}\t{}\t{}\t{}\t{}\t{}'.format(
                                dropout_rate, fit_iter+2, epoch,
                    xgb1_var, xgb2_var, xgb3_var, train_mse, val_mse),
                    file=results_file, flush=True)
                if epoch < 50:
                    model.fit(X_train, Y_train, epochs=5, verbose=1)
                    epoch += 5
                elif epoch < max_epoch:
                    model.fit(X_train, Y_train, epochs=25, verbose=0)
                    epoch += 25

Building model with sigmoid activation and 0.000 dropout
0.0 0 0

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

3.011575e-06 1.1730004e-08 6.967876e-09 1.3417462996397012 3.127217302685323
Train on 13003 samples
Epoch 1/5
13003/13003 [==============================] - 1s 89us/sample - loss: 0.7233 - mse: 0.7233
Epoch 2/5
13003/13003 [==============================] - 1s 74us/sample - loss: 0.6097 - mse: 0.6097
Epoch 3/5
13003/13003 [==============================] - 1s 55us/sample - loss: 0.6005 - mse: 0.6005
Epoch 4/5
13003/13003 [=================

0.0 0 425
0.9816805 0.19967252 0.11020377 0.016204562627547422 0.30108086872479617
0.0 0 450
1.0455962 0.20893724 0.09539155 0.0178087516960296 0.2982684877846089
0.0 0 475
1.0539359 0.22104819 0.095842905 0.014894483634738182 0.293503205450913
Building model with sigmoid activation and 0.050 dropout
0.05 0 0

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

1.441074e-06 1.003799e-08 4.4356963e-09 1.2319988541270008 1.8704885495935852
Train on 13003 samples
Epoch 1/5
13003/13003 [==============================] - 1s 78us/sample - loss: 0.7429 - mse: 0.

0.05 0 375
0.8163718 0.14618123 0.06308964 0.03824791389239821 0.3293885365171417
0.05 0 400
0.7220243 0.16150023 0.052256644 0.04540639804750721 0.4273269518197794
0.05 0 425
0.77184695 0.12386516 0.061992053 0.039552473413843964 0.3841679527187933
0.05 0 450
0.88013774 0.14197761 0.07372415 0.036091498061698854 0.3461116640019691
0.05 0 475
0.8295229 0.14727496 0.05047323 0.03561896575091609 0.3507056557561654
Building model with sigmoid activation and 0.125 dropout
0.125 0 0

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

2.3332777e-06 1.0930964e-

0.125 0 300
0.70992893 0.114390485 0.07970229 0.1834216517765748 0.6774035999036083
0.125 0 325
0.7621325 0.11456015 0.078350134 0.17993744583317553 0.5909834298512652
0.125 0 350
0.71619433 0.11840513 0.07425235 0.19250374012401816 0.6568881330187369
0.125 0 375
0.71135294 0.14237176 0.07386554 0.16530277036406404 0.5873056437915629
0.125 0 400
0.7660814 0.13639398 0.072346345 0.19399562017962219 0.6317327064262556
0.125 0 425
0.6963667 0.117155865 0.07071668 0.16001606341559366 0.6369805237682861
0.125 0 450
0.70636314 0.13004535 0.08671193 0.21166386964384099 0.6934059903676455
0.125 0 475
0.72856206 0.12457312 0.071034074 0.16225657531843526 0.6186013839351883
Building model with sigmoid activation and 0.250 dropout
0.25 0 0

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.kera

0.25 0 225
0.7162065 0.14236598 0.07743611 0.5767659030503072 1.2196838770212317
0.25 0 250
0.647318 0.12221185 0.065250404 0.5529604450479593 1.1661852942793125
0.25 0 275
0.7748517 0.14183578 0.08381873 0.5968756422139337 1.163538039423589
0.25 0 300
0.6710045 0.1223993 0.07153831 0.5260033846798361 1.1785643397992849
0.25 0 325
0.70502883 0.14518121 0.07325665 0.590421805557008 1.26856051094355
0.25 0 350
0.6642938 0.122583196 0.07409686 0.5173498083875392 1.1329275408442785
0.25 0 375
0.68450254 0.122221015 0.07653083 0.5657934043466434 1.1728435305774099
0.25 0 400
0.5882972 0.11621595 0.059905503 0.512833912816514 1.1965067671496192
0.25 0 425
0.6476149 0.13325807 0.061747607 0.5481885297526371 1.1852802096557402
0.25 0 450
0.7120792 0.15786698 0.07283409 0.6053131285906173 1.2079669978510819
0.25 0 475
0.71980053 0.16009563 0.06726426 0.5451415426930764 1.1492674097310263
Building model with sigmoid activation and 0.375 dropout
0.375 0 0

If you intended to run this layer in flo

...

13003/13003 [==============================] - 1s 63us/sample - loss: 0.5077 - mse: 0.5077
Epoch 2/5
13003/13003 [==============================] - 1s 53us/sample - loss: 0.5130 - mse: 0.5130
Epoch 3/5
13003/13003 [==============================] - 1s 54us/sample - loss: 0.4988 - mse: 0.4988
Epoch 4/5
13003/13003 [==============================] - 1s 56us/sample - loss: 0.4843 - mse: 0.4843
Epoch 5/5
13003/13003 [==============================] - 1s 54us/sample - loss: 0.5047 - mse: 0.5047
0.375 1 45
0.49240395 0.05605495 0.014223962 0.6206147459033018 1.4146039754671023
Train on 13003 samples
Epoch 1/5
13003/13003 [==============================] - 1s 52us/sample - loss: 0.5020 - mse: 0.5020
Epoch 2/5
13003/13003 [==============================] - 1s 59us/sample - loss: 0.4952 - mse: 0.4952
Epoch 3/5
13003/13003 [==============================] - 1s 72us/sample - loss: 0.4863 - mse: 0.4863
Epoch 4/5
13003/13003 [==============================] - 1s 67us/sample - loss: 0.4810 - mse: 0.

13003/13003 [==============================] - 1s 58us/sample - loss: 0.0651 - mse: 0.0651
Epoch 4/5
13003/13003 [==============================] - 1s 50us/sample - loss: 0.0657 - mse: 0.0657
Epoch 5/5
13003/13003 [==============================] - 1s 46us/sample - loss: 0.0654 - mse: 0.0654
0.0 2 40
1.00577 0.16383311 0.07212142 0.07420632901489774 0.22412673319901913
Train on 13003 samples
Epoch 1/5
13003/13003 [==============================] - 1s 51us/sample - loss: 0.0624 - mse: 0.0624
Epoch 2/5
13003/13003 [==============================] - 1s 54us/sample - loss: 0.0639 - mse: 0.0639
Epoch 3/5
13003/13003 [==============================] - 1s 57us/sample - loss: 0.0615 - mse: 0.0615
Epoch 4/5
13003/13003 [==============================] - 1s 51us/sample - loss: 0.0620 - mse: 0.0620
Epoch 5/5
13003/13003 [==============================] - 1s 45us/sample - loss: 0.0620 - mse: 0.0620
0.0 2 45
0.8559191 0.18411006 0.047311287 0.05919696568826775 0.25910245430674117
Train on 13003 sam

13003/13003 [==============================] - 1s 50us/sample - loss: 0.1622 - mse: 0.1622
Epoch 2/5
13003/13003 [==============================] - 1s 51us/sample - loss: 0.1592 - mse: 0.1592
Epoch 3/5
13003/13003 [==============================] - 1s 59us/sample - loss: 0.1631 - mse: 0.1631
Epoch 4/5
13003/13003 [==============================] - 1s 58us/sample - loss: 0.1505 - mse: 0.1505
Epoch 5/5
13003/13003 [==============================] - 1s 57us/sample - loss: 0.1619 - mse: 0.1619
0.05 2 40
0.9596597 0.12287791 0.07009942 0.10218503786319924 0.3460709509313739
Train on 13003 samples
Epoch 1/5
13003/13003 [==============================] - 1s 55us/sample - loss: 0.1520 - mse: 0.1520
Epoch 2/5
13003/13003 [==============================] - 1s 55us/sample - loss: 0.1496 - mse: 0.1496
Epoch 3/5
13003/13003 [==============================] - 1s 51us/sample - loss: 0.1556 - mse: 0.1556
Epoch 4/5
13003/13003 [==============================] - 1s 49us/sample - loss: 0.1499 - mse: 0.14

13003/13003 [==============================] - 1s 72us/sample - loss: 0.4041 - mse: 0.4041
Epoch 2/5
13003/13003 [==============================] - 1s 57us/sample - loss: 0.4073 - mse: 0.4073
Epoch 3/5
13003/13003 [==============================] - 1s 70us/sample - loss: 0.3971 - mse: 0.3971
Epoch 4/5
13003/13003 [==============================] - 1s 54us/sample - loss: 0.3960 - mse: 0.3960
Epoch 5/5
13003/13003 [==============================] - 1s 54us/sample - loss: 0.3914 - mse: 0.3914
0.25 2 35
0.74932235 0.097393475 0.032951783 0.45773534395961185 1.1443252311357195
Train on 13003 samples
Epoch 1/5
13003/13003 [==============================] - 1s 49us/sample - loss: 0.3971 - mse: 0.3971
Epoch 2/5
13003/13003 [==============================] - 1s 52us/sample - loss: 0.3892 - mse: 0.3892
Epoch 3/5
13003/13003 [==============================] - 1s 52us/sample - loss: 0.3791 - mse: 0.3791
Epoch 4/5
13003/13003 [==============================] - 1s 52us/sample - loss: 0.3839 - mse: 0

13003/13003 [==============================] - 1s 54us/sample - loss: 0.6799 - mse: 0.6799
Epoch 4/5
13003/13003 [==============================] - 1s 48us/sample - loss: 0.6690 - mse: 0.6690
Epoch 5/5
13003/13003 [==============================] - 1s 49us/sample - loss: 0.6682 - mse: 0.6682
0.5 2 30
0.21924996 0.032896604 0.0020571484 0.6068930908186204 1.338918746899063
Train on 13003 samples
Epoch 1/5
13003/13003 [==============================] - 1s 51us/sample - loss: 0.6642 - mse: 0.6642
Epoch 2/5
13003/13003 [==============================] - 1s 48us/sample - loss: 0.6797 - mse: 0.6797
Epoch 3/5
13003/13003 [==============================] - 1s 50us/sample - loss: 0.6706 - mse: 0.6706
Epoch 4/5
13003/13003 [==============================] - 1s 48us/sample - loss: 0.6616 - mse: 0.6616
Epoch 5/5
13003/13003 [==============================] - 1s 48us/sample - loss: 0.6686 - mse: 0.6686
0.5 2 35
0.24121062 0.034756828 0.0019475244 0.6177896142821553 1.328066307477826
Train on 13003 